Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# 00. Installation and configuration
This notebook configures your library of notebooks to connect to an Azure Machine Learning Workspace.  In this case, a library contains all of the notebooks in the current folder and any nested folders.  You can configure this notebook to use an existing workspace or create a new workspace.

## What is an Azure ML Workspace and why do I need one?

An AML Workspace is an Azure resource that organizes and coordinates the actions of many other Azure resources to assist in executing and sharing machine learning workflows.  In particular, an AML Workspace coordinates storage, databases, and compute resources providing added functionality for machine learning experimentation, operationalization, and the monitoring of operationalized models.

## Prerequisites

### 1. Access Azure Subscription

In order to create an AML Workspace, first you need access to an Azure Subscription.  You can [create your own](https://azure.microsoft.com/en-us/free/) or get your existing subscription information from the [Azure portal](https://portal.azure.com).

### 2. If you're running on your own local environment, install Azure ML SDK and other libraries

If you are running in your own environment, follow [SDK installation instructions](https://docs.microsoft.com/azure/machine-learning/service/how-to-configure-environment).  If you are running in Azure Notebooks or another Microsoft managed environment, the SDK is already installed.

Also install following libraries to your environment. Many of the example notebooks depend on them

```
(myenv) $ conda install -y matplotlib tqdm scikit-learn
```

Once installation is complete, check the Azure ML SDK version:

In [ ]:
import azureml.core

print("SDK Version:", azureml.core.VERSION)

### 3. Make sure your subscription is registered to use ACI
Azure Machine Learning makes use of Azure Container Instance (ACI). You need to ensure your subscription has been registered to use ACI in order be able to deploy a dev/test web service.  If you have run through the quickstart experience you have already performed this step.  Otherwise you will need to use the [Azure CLI](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli?view=azure-cli-latest) and execute the following commands.

```shell
# check to see if ACI is already registered
(myenv) $ az provider show -n Microsoft.ContainerInstance -o table

# if ACI is not registered, run this command.
# note you need to be the subscription owner in order to execute this command successfully.
(myenv) $ az provider register -n Microsoft.ContainerInstance
```

## Set up your Azure Machine Learning workspace

### Option 1: You have workspace already
If you ran the Azure Machine Learning [quickstart](https://docs.microsoft.com/en-us/azure/machine-learning/service/quickstart-get-started) in Azure Notebooks, you already have a configured workspace!  You can go to your Azure Machine Learning Getting Started library, view *config.json* file, and copy-paste the values for subscription ID, resource group and workspace name below.

If you have a workspace created another way, [these instructions](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-environment#create-workspace-configuration-file) describe how to get your subscription and workspace information.

If this cell succeeds, you're done configuring this library!  Otherwise continue to follow the instructions in the rest of the notebook.

In [ ]:
import os

subscription_id = os.environ.get("SUBSCRIPTION_ID", "<my-subscription-id>")
resource_group = os.environ.get("RESOURCE_GROUP", "<my-resource-group>")
workspace_name = os.environ.get("WORKSPACE_NAME", "<my-workspace-name>")

In [ ]:
from azureml.core import Workspace

try:
   ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
   ws.write_config()
   print('Workspace configuration succeeded. You are all set!')
except:
   print('Workspace not found. Run the cells below.')

### Option 2: You don't have workspace yet


#### Requirements

Inside your Azure subscription, you will need access to a _resource group_, which organizes Azure resources and provides a default region for the resources in a group.  You can see what resource groups to which you have access, or create a new one in the [Azure portal](https://portal.azure.com).  If you don't have a resource group, the create workspace command will create one for you using the name you provide.

To create or access an Azure ML Workspace, you will need to import the AML library and the following information:
* A name for your workspace
* Your subscription id
* The resource group name

**Note**: As with other Azure services, there are limits on certain resources (for eg. BatchAI cluster size) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

#### Supported Azure Regions
Specify a region where your workspace will be located from the list of [Azure Machine Learning regions](https://linktoregions)

In [ ]:
import os

subscription_id = os.environ.get("SUBSCRIPTION_ID", "<my-subscription-id>")
resource_group = os.environ.get("RESOURCE_GROUP", "my-aml-resource-group")
workspace_name = os.environ.get("WORKSPACE_NAME", "my-first-workspace")

workspace_region = os.environ.get("WORKSPACE_REGION", "eastus2")

#### Create the workspace
This cell will create an AML workspace for you in a subscription provided you have the correct permissions.

This will fail when:
1. You do not have permission to create a workspace in the resource group
2. You do not have permission to create a resource group if it's non-existing.
2. You are not a subscription owner or contributor and no Azure ML workspaces have ever been created in this subscription

If workspace creation fails, please work with your IT admin to provide you with the appropriate permissions or to provision the required resources.

In [ ]:
# import the Workspace class and check the azureml SDK version
from azureml.core import Workspace

ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,
                      create_resource_group = True,
                      exist_ok = True)
ws.get_details()
ws.write_config()

## Create compute resources for your training experiments

Many of the subsequent examples use BatchAI clusters to train models at scale. To create a **CPU** cluster now, run the cell below. The autoscale settings mean that the cluster will scale down to 0 nodes when inactive and up to 8 nodes when busy.

In [ ]:
from azureml.core.compute import ComputeTarget, BatchAiCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = BatchAiCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                                autoscale_enabled=True,
                                                                cluster_min_nodes=0, 
                                                                cluster_max_nodes=6)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

To create a **GPU** cluster, run the cell below. Note that your subscription must have sufficient quota for GPU VMs or the command will fail. To increase quota, see [these instructions](https://docs.microsoft.com/en-us/azure/azure-supportability/resource-manager-core-quotas-request). 

In [ ]:
from azureml.core.compute import ComputeTarget, BatchAiCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your GPU cluster
gpu_cluster_name = "gpucluster"

# Check if cluster exists already
try:
    gpu_cluster = ComputeTarget(workspace=ws, name=gpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = BatchAiCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                                autoscale_enabled=True,
                                                                cluster_min_nodes=0, 
                                                                cluster_max_nodes=6)
    gpu_cluster = ComputeTarget.create(ws, gpu_cluster_name, compute_config)

gpu_cluster.wait_for_completion(show_output=True)

## Success!
Great, you are ready to move on to the rest of the sample notebooks. A good place to start is the [01.train-model tutorial](./tutorials/01.train-model.ipynb) to learn how to train and then deploy an image classification model.